## Align utterance in Memor to open subtitle parallel bitext

In [1]:
import json
from lxml import etree
import lxml
from tqdm import tqdm

#### Load MEMOR Dataset

In [2]:
# Load Memor Dataset
with open('memor/data.json') as f:
    data = json.load(f)

In [46]:
episode = []
# Collect memor data from the same episode
for item in data:
    if item.strip().split('_')[0]=='S01E02':
        episode.append(data[item])

In [47]:
# Split sentence into small segments based on , and use segments to search the open subtitle corpus
segments = []
for x in episode:
    utts = x['sentences']
    for utt in utts:
        temp = utt.strip().split(',')
        for segment in temp:
            if len(segment.split(" "))<6:
                pass
                # print('====', segment)
            else:
                print(segment)
                segments.append(segment)
    print('=='*50)

hey raj!  still not talking to me
 he can't talk to women.
he can't talk to attractive women
 or in your case a cheesecake-scented goddess!
 there's gonna be some furniture delivered?
 let's assume that they can. lois lane is falling
 accelerating at an initial rate of 32 feet per second per second. superman swoops down to save her by reaching out two arms of steel. miss lane
 who is now travelling at approximately 120 miles per hour
 and is immediately sliced into three equal pieces.
 it is well established that superman's flight is a feat of strength
 it is an extension of his ability to leap tall buildings
 an ability he derives from earth's yellow sun.
 and you don't have a problem with that
 how does he fly at night.
 a combination of the moon's solar reflection and the energy storage capacity of kryptonian skin cells.
 a combination of the moon's solar reflection and the energy storage capacity of kryptonian skin cells.
i'm just going to go wash up.
i have 26 hundred comic books 

In [24]:
print(len(segments))

149


#### Load Open Subtitle BiText Corpus

In [3]:
# Load Open Subtitle Bitext
xml_file = etree.parse('../open_subtitle/en-zh_cn.tmx')

In [36]:
# Collect subtitles
en_subtitle = []
zh_subtitle = []
root = xml_file.getroot()
count = 0
for node in tqdm(root.iter()):
    if node.tag=='seg':
        if count % 2 ==0:
            en_subtitle.append(node.text)
        else:
            zh_subtitle.append(node.text)
        count += 1

49845028it [00:35, 1398557.88it/s]


In [39]:
# Check the processed subtitles
print(len(en_subtitle))
print(len(zh_subtitle))
for i in range(10):
    print(en_subtitle[i])
    print(zh_subtitle[i])
    print('=='*50)
for i in range(10):
    print(en_subtitle[-i])
    print(zh_subtitle[-i])
    print('=='*50)

Ah, this is greasy. I want to eat kimchee.
好想要吃泡菜
Is Chae Yoon's coordinator in here?
崔允的造型师在吗
Excuse me, aren't you Chae Yoon's coordinator?
请问一下 你是不是崔允的造型师
Yes. Me?
我吗
-Chae Yoon is done singing.
崔允已经唱完了
This lady right next to me... everyone knows who she is right?
这位小姐是谁 我想大家都知道吧
She is my forever idol...she is my rival whom I can never win against, she is the one who always makes me happy, and she is my mother.
她是我永远的偶像 也是我无法击倒的对手 她是令我怦然心动的恋人
Everyone should remember my mom as one of the greatest movie stars, but instead of becoming an actress, she wanted to become a singer just like me.
也许在各位的记忆里 我母亲 她是曾经红极一时的女演员
When I was stressed and exhausted, she'd always sing me a song while having me on her lap.
而是像我一样当歌手 当我感到累还有疲倦的时候 每一次她都让我
Today, I would like to share that moment with everyone.
然后为我唱一首歌 今天我想要 特别和各位
Ah, this is greasy. I want to eat kimchee.
好想要吃泡菜
Subtitle translation by Brian Athey
字幕翻译：
MY LADIES BY SHIN MICHIMA
（我的女人们）
It's because... that's the way I want it.
这是我自己的

#### Align the MRMOR Corpus with the Open Subtitle Corpus

In [48]:
print(len(segments))

101


In [49]:
for segment in segments:
    for i, subtitle in enumerate(en_subtitle):
        if segment in subtitle:
            print("Index:", i)
            print("Target_en:", segment)
            print("Target_zh:", zh_subtitle[i])
            print("Context:")
            for b in range(3):
                print(en_subtitle[i+2-b], zh_subtitle[i+2-b])
            print('=='*50)

Index: 4236022
Target_en:  and is immediately sliced into three equal pieces.
Target_zh: 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分
Context:
In what space, sir? In what space? 哪还有时间 先生?
Unless Superman matches her speed and decelerates. 除非超人赶上她的速度并减速
Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces. 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分
Index: 4236085
Target_en:  because the evidence suggests the coffee table's having a tiny garage sale.
Target_zh: 现在 我只是假设这是沙发 因为证据显示 咖啡桌上正进行一场小型旧货出售
Context:
No. 没有想过
Did it occur to you that not everyone has the compulsive need to sort organize and label the entire world around them? 你有没想过 并不是每个人都觉得必须要... 归类 整理 并将一切事物都贴上标签?
I'm just inferring that this as a couch because the evidence suggests the coffee table's having a tiny garage sale. 现在 我只是假设这是沙发 因为证据显示 咖啡桌上正进行一场小型旧货出售
Index: 3498567
Target_en:  i just want you to know that
Target_zh: 大卫 我只要你知道我要走了
Context:
i told you Da